# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [23]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import FreqDist

In [2]:
# your code here
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [3]:
df.columns = ['target','id','date','flag','user','text']

In [4]:
df = df.sample(n=20000, random_state=42)

In [12]:
df

,target,id,date,flag,user,text
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick"
...,...,...,...,...,...,...
1374482,4,2051447103,Fri Jun 05 22:02:36 PDT 2009,NO_QUERY,_Jaska,@girlwonder24 Thanks.
667014,0,2245469948,Fri Jun 19 16:10:39 PDT 2009,NO_QUERY,julianicolao,"trying to study for the biggest test, next wee..."
1451234,4,2063022808,Sun Jun 07 01:05:46 PDT 2009,NO_QUERY,ElaineToni,Just finished watching Your Song Presents: Boy...
1181412,4,1982082859,Sun May 31 10:29:36 PDT 2009,NO_QUERY,lindseyrd20,@janfran813 awww i can't wait to get one


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [20]:
def clean_up(s):
    # Remove URLs
    s = re.sub(r'http\S+', '', s)
    
    # Replace special characters and numbers with whitespace
    s = re.sub(r'[^a-zA-Z\s]', ' ', s)
    
    # Remove extra whitespaces
    s = ' '.join(s.split())

    return s.lower()

def tokenize(s):

    nltk.download('punkt', quiet=True)
    
    # Use NLTK's word_tokenize function
    words = word_tokenize(s)

    return words

def get_wordnet_pos(tag):
    """Map POS tag to first character lemmatize() accepts."""
    tag = tag[0].upper()
    tag_dict = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    return tag_dict.get(tag, wordnet.NOUN)

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    # Initialize stemmer and lemmatizer
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Perform stemming and lemmatization
    stemmed_and_lemmatized = []
    for word in l:
        # Stemming
        stemmed_word = stemmer.stem(word)

        # Lemmatization
        pos_tag = nltk.pos_tag([word])[0][1]
        pos_tag_for_lemmatization = get_wordnet_pos(pos_tag)
        lemmatized_word = lemmatizer.lemmatize(stemmed_word, pos=pos_tag_for_lemmatization)
        stemmed_and_lemmatized.append((lemmatized_word))
    return stemmed_and_lemmatized

# Test the function with the tokenized result from the previous step
tokenized_result = ['ironhack', 's', 'q', 'website', 'is']
stem_and_lemmatized_result = stem_and_lemmatize(tokenized_result)
print(stem_and_lemmatized_result)

def remove_stopwords(l):
    # Get the English stopwords
    stop_words = set(stopwords.words('english'))

    # Remove stopwords from the list
    filtered_list = [word for word in l if word not in stop_words]

    return filtered_list

['ironhack', 's', 'q', 'websit', 'be']


In [21]:
# Apply the functions to the whole DataFrame
df['text_processed'] = df['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)

In [22]:
# Display the DataFrame with the new 'text_processed' column
df

,target,id,date,flag,user,text,text_processed
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...,"[nkluvr, eva, poor, littl, dumpl, holmdel, vid..."
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...,"[bed, get, ta, wake, hella, earli, tomorrow, m..."
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...,"[havent, abl, listen, yet, speaker, bust]"
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,"[rememb, whi, solv, rel, big, equat, two, unkn..."
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick","[ate, much, feel, sick]"
...,...,...,...,...,...,...,...
1374482,4,2051447103,Fri Jun 05 22:02:36 PDT 2009,NO_QUERY,_Jaska,@girlwonder24 Thanks.,"[girlwond, thank]"
667014,0,2245469948,Fri Jun 19 16:10:39 PDT 2009,NO_QUERY,julianicolao,"trying to study for the biggest test, next wee...","[tri, studi, big, test, next, week, work, thin..."
1451234,4,2063022808,Sun Jun 07 01:05:46 PDT 2009,NO_QUERY,ElaineToni,Just finished watching Your Song Presents: Boy...,"[finish, watch, song, present, boystown]"
1181412,4,1982082859,Sun May 31 10:29:36 PDT 2009,NO_QUERY,lindseyrd20,@janfran813 awww i can't wait to get one,"[janfran, awww, wait, get, one]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [25]:
# your code here
words = [word for sublist in df['text_processed'] for word in sublist]

# Calculate the frequency distribution
freq_distribution = FreqDist(words)

# Select the top 5,000 words
top_words = freq_distribution.most_common(5000)

# Display the top words
top_words

[('get', 2259),
 ('go', 1996),
 ('wa', 1400),
 ('day', 1326),
 ('good', 1189),
 ('thi', 1155),
 ('work', 1122),
 ('love', 1116),
 ('like', 1030),
 ('quot', 941),
 ('well', 851),
 ('today', 829),
 ('time', 795),
 ('thank', 784),
 ('u', 784),
 ('one', 747),
 ('lol', 741),
 ('know', 724),
 ('miss', 717),
 ('want', 711),
 ('make', 710),
 ('see', 690),
 ('im', 681),
 ('back', 664),
 ('feel', 657),
 ('think', 648),
 ('realli', 605),
 ('night', 582),
 ('new', 569),
 ('amp', 568),
 ('still', 549),
 ('oh', 540),
 ('need', 538),
 ('watch', 536),
 ('come', 534),
 ('look', 523),
 ('na', 514),
 ('ha', 513),
 ('home', 507),
 ('hope', 496),
 ('say', 484),
 ('much', 480),
 ('last', 450),
 ('great', 449),
 ('morn', 445),
 ('take', 424),
 ('wait', 423),
 ('tomorrow', 421),
 ('twitter', 419),
 ('wish', 416),
 ('bad', 399),
 ('sleep', 382),
 ('haha', 379),
 ('whi', 375),
 ('follow', 369),
 ('sad', 366),
 ('veri', 352),
 ('fun', 348),
 ('would', 342),
 ('nice', 336),
 ('sorri', 332),
 ('happi', 330),
 ('ri

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [27]:
# your code here


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [1]:
# your code here

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here